In [1]:
from Parser import Parser
from ipycanvas import Canvas
from typing import Dict, Callable, Tuple, List
import ipywidgets as widgets
import numpy as np
import math

MOVE_ACTION = 'move'
ZOOM_ACTION = 'zoom'
SELECT_ACTION = 'select'

def defaultColorMap(cells, variables):
    return [
        ('#004400', 'green', cells[variables['cycle_model']] == 5),
        ('brown', 'black', cells[variables['cycle_model']] == 100),
        ('#440000', 'red', cells[variables['cycle_model']] == 101)
    ]

def count(array):
    dic = {}
    for num in array:
        if not num in dic:
            dic[num] = 0
        dic[num] += 1
    return dic
    

class Interactor2D:
    def __init__(self, parser, width: int = 500, height: int = 400, colorMap = defaultColorMap):
        self._currentFrame = parser.getFrameRange()[0]
        self._canvas = Canvas(width=width, height=height)
        self._parser = parser
        self._colorMap = colorMap
        self._height = height
        self._width = width
        
        frame = parser.getFrame(self._currentFrame)
        mesh = frame.environment.mesh
        self._zoom = max((mesh.boundsX[1] - mesh.boundsX[0]) / width, (mesh.boundsY[1] - mesh.boundsY[0]) / height)
        
        self._xOffset = mesh.boundsX[0]
        self._yOffset = mesh.boundsY[0]
        self._selectedCell = None
        
        self._clicking = False
        self._dragStartX = 0
        self._dragStartY = 0
        self._actionOriginX = 0
        self._actionOriginY = 0
        
        self._canvas.on_mouse_down(self.onMouseDown)
        self._canvas.on_mouse_move(self.onMouseMove)
        self._canvas.on_mouse_up(self.onMouseUp)
        self._canvas.on_mouse_out(self.onMouseOut)
        
        self.action = MOVE_ACTION
        
        self._buttons = widgets.RadioButtons(
            options=[MOVE_ACTION, ZOOM_ACTION, SELECT_ACTION],
            value=self.action,
            description='Mouse Action:',
            disabled=False,
        )
        self._buttons.observe(self.onToolChange, names='value')
        
        self._frameSelector = widgets.IntSlider(
            value=self._currentFrame,
            min=parser.getFrameRange()[0],
            max=parser.getFrameRange()[1] - 1,
            step=1,
            description='Frame:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self._frameSelector.observe(self.onFrameChange, names='value')
        
        self.update()
    
    def update(self):
        self._canvas.clear()
        
        canvas = self._canvas
        
        frame = self._parser.getFrame(self._currentFrame)
    
        cells = frame.cells.data
        cellVariables = frame.cells.variables
        
        canvas.stroke_style = 'blue'

        x = (cells[cellVariables['position.x']] - self._xOffset) / self._zoom
        y = (cells[cellVariables['position.y']] - self._yOffset) / self._zoom
        r = self.radiusOfCells(cells, cellVariables) / self._zoom
        
        combined = np.array([x, y, r])
        
        for fill, stroke, indices in self._colorMap(cells, cellVariables):
            split = combined[:,indices]
            if split.shape[0] == 0:
                continue
            x, y, r = split
            canvas.fill_style = fill
            canvas.fill_circles(x, y, r)
            canvas.stroke_style = stroke
            canvas.stroke_circles(x, y, r)
            
        
        canvas.fill_style = '#A0A0A0'
        canvas.font = '10px serif'
        canvas.fill_text(f"(x: {self._xOffset}, y: {self._yOffset}, s: {self._zoom}, c: {self._selectedCell})", 10, self._height - 10)
        
    
    def onMouseDown(self, x: int, y: int):
        self._dragStartX = x
        self._dragStartY = y
        self._envActionOriginX = x * self._zoom + self._xOffset
        self._envActionOriginY = y * self._zoom + self._yOffset
        self._actionOriginX = x
        self._actionOriginY = y
        self._actionOriginZoom = self._zoom
        
        if self.action == SELECT_ACTION:
            self.selectCell(self._envActionOriginX, self._envActionOriginY)
        else:
            self._clicking = True
    
    def onMouseUp(self, x: int, y: int):
        self._clicking = False
    
    def onMouseOut(self, x: int, y: int):
        self._clicking = False
    
    def onMouseMove(self, x: int, y: int):
        if self._clicking:
            if self.action == MOVE_ACTION:
                self._xOffset -= (x - self._dragStartX) * self._zoom
                self._yOffset -= (y - self._dragStartY) * self._zoom
            elif self.action == ZOOM_ACTION:
                self._zoom = max(self._actionOriginZoom * 2 ** ((self._actionOriginY - y) / 25), 0.0001)
                self._xOffset = self._envActionOriginX - self._actionOriginX * self._zoom
                self._yOffset = self._envActionOriginY - self._actionOriginY * self._zoom
                
            self._dragStartX = x
            self._dragStartY = y
            self.update()
    
    def onToolChange(self, action):
        self.action = action.new
    
    def onFrameChange(self, action):
        self._currentFrame = action.new
        self.update()
    
    def selectCell(self, x: float, y: float):
        frame = self._parser.getFrame(self._currentFrame)
    
        cells = frame.cells.data
        cellVariables = frame.cells.variables
        
        distances = np.sqrt(np.square(x - cells[cellVariables['position.x']]) + np.square(y - cells[cellVariables['position.y']])) - self.radiusOfCells(cells, cellVariables)
        minIndex = np.argmin(distances)
        
        if distances[minIndex] <= 0:
            self._selectedCell = cells[cellVariables['ID'], minIndex]
            self.update()
        
        
 
    def radiusOfCells(self, cells, variables):
        return (cells[variables['total_volume']] * (3 / ( 4 * math.pi))) ** (1 / 3)

    def show(self):
        display(self._canvas, self._buttons, self._frameSelector)
            



def InteractiveEnvironment(outputPath: str, width: int = 500, height: int = 400):
    parser = Parser(outputPath)
    if parser.getFrame(parser.getFrameRange()[0]).environment.is2D:
        return Interactor2D(parser, width, height)

In [2]:
env = InteractiveEnvironment('./sample-output', width=800, height=800)
env.show()

Canvas(height=800, width=800)

RadioButtons(description='Mouse Action:', options=('move', 'zoom', 'select'), value='move')

IntSlider(value=0, continuous_update=False, description='Frame:', max=9)

In [3]:
help(Canvas)

Help on class Canvas in module ipycanvas.canvas:

class Canvas(_CanvasBase)
 |  Canvas(*args, **kwargs)
 |  
 |  Create a Canvas widget.
 |  
 |  Args:
 |      width (int): The width (in pixels) of the canvas
 |      height (int): The height (in pixels) of the canvas
 |      caching (boolean): Whether commands should be cached or not
 |  
 |  Method resolution order:
 |      Canvas
 |      _CanvasBase
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Create a Canvas widget.
 |  
 |  __setattr__(self, name, value)
 |      Implement setattr(self, name, value).
 |  
 |  arc(self, x, y, radius, start_angle, end_angle, anticlockwise=False)
 |      Add a circular arc centered at ``(x, y)`` with a radius of ``radius`` to th

In [4]:
env.show()

Canvas(height=800, width=800)

RadioButtons(description='Mouse Action:', options=('move', 'zoom', 'select'), value='move')

IntSlider(value=0, continuous_update=False, description='Frame:', max=9)

In [5]:
math.sig(-1)

AttributeError: module 'math' has no attribute 'sig'

In [6]:
int(False) * 2 - 1

-1

In [7]:
import itk

from itkwidgets import view

In [8]:
help(itk)

Help on package itk:

NAME
    itk

PACKAGE CONTENTS
    ITKAnisotropicSmoothingPython
    ITKAntiAliasPython
    ITKBiasCorrectionPython
    ITKBinaryMathematicalMorphologyPython
    ITKBridgeNumPyPython
    ITKClassifiersPython
    ITKColormapPython
    ITKCommonBasePython
    ITKCommonPython
    ITKConnectedComponentsPython
    ITKConvolutionPython
    ITKCostFunctionsPython
    ITKCurvatureFlowPython
    ITKDeconvolutionPython
    ITKDisplacementFieldPython
    ITKDistanceMapPython
    ITKEigenPython
    ITKFFTPython
    ITKFastMarchingBasePython
    ITKFastMarchingPython
    ITKFiniteDifferencePython
    ITKIOBMPPython
    ITKIOBioRadPython
    ITKIOBrukerPython
    ITKIOGDCMPython
    ITKIOGEPython
    ITKIOGIPLPython
    ITKIOHDF5Python
    ITKIOIPLPython
    ITKIOImageBaseBasePython
    ITKIOImageBasePython
    ITKIOJPEG2000Python
    ITKIOJPEGPython
    ITKIOLSMPython
    ITKIOMINCPython
    ITKIOMRCPython
    ITKIOMeshBYUPython
    ITKIOMeshBasePython
    ITKIOMeshFreeSurferP

In [10]:
import numpy as np

from itkwidgets import view

from Parser import Parser

parser = Parser('./sample-output')

cells = parser.getFrame(0).cells


print(type(cells.data))

v = view(point_sets=[cells.data[1:4].T])
v

<class 'numpy.ndarray'>


Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [8]:
import pyvista as pv
from Parser import Parser

parser = Parser('./sample-output')
cells = parser.getFrame(0).cells

# mesh = pyvista.Sphere()
pl = pv.PlotterITK()  
pl.add_points(cells.data[1:4].T)
pl.show()

Viewer(geometries=[], point_set_colors=array([[1., 1., 1.]], dtype=float32), point_set_opacities=array([1.], d…

Viewer(geometries=[], point_set_colors=array([[1., 1., 1.]], dtype=float32), point_set_opacities=array([1.], d…

In [1]:
import vtk

dir(vtk)

['EnsightReaderCellIdMode',
 'IMPLICIT_STRUCTURED_MODE',
 'NON_SPARSE_MODE',
 'SINGLE_PROCESS_MODE',
 'SPARSE_MODE',
 'VTKIS_ACTOR',
 'VTKIS_ANIM_OFF',
 'VTKIS_ANIM_ON',
 'VTKIS_CAMERA',
 'VTKIS_CLIP',
 'VTKIS_DOLLY',
 'VTKIS_ENV_ROTATE',
 'VTKIS_EXIT',
 'VTKIS_FORWARDFLY',
 'VTKIS_GESTURE',
 'VTKIS_IMAGE2D',
 'VTKIS_IMAGE3D',
 'VTKIS_IMAGE_SLICING',
 'VTKIS_JOYSTICK',
 'VTKIS_LOAD_CAMERA_POSE',
 'VTKIS_MENU',
 'VTKIS_NONE',
 'VTKIS_PAN',
 'VTKIS_PICK',
 'VTKIS_POSITION_PROP',
 'VTKIS_REVERSEFLY',
 'VTKIS_ROTATE',
 'VTKIS_SLICE',
 'VTKIS_SPIN',
 'VTKIS_START',
 'VTKIS_TIMER',
 'VTKIS_TOGGLE_DRAW_CONTROLS',
 'VTKIS_TRACKBALL',
 'VTKIS_TWO_POINTER',
 'VTKIS_USCALE',
 'VTKIS_USERINTERACTION',
 'VTKIS_WINDOW_LEVEL',
 'VTKIS_ZOOM',
 'VTKI_MAX_POINTERS',
 'VTKI_TIMER_FIRST',
 'VTKI_TIMER_UPDATE',
 'VTKKW_FPMM_SHIFT',
 'VTKKW_FP_MASK',
 'VTKKW_FP_SCALE',
 'VTKKW_FP_SHIFT',
 'VTK_21_POINT_WEDGE',
 'VTK_3D_EXTENT',
 'VTK_ABS',
 'VTK_ACCUMULATION_MODE_MAX',
 'VTK_ACCUMULATION_MODE_MIN',
 'VTK_AC

In [80]:
import vtk

from ipywidgets import Image, Output

# from IPython.display import Image
from Parser import Parser
from ipycanvas import Canvas
from typing import Dict, Callable, Tuple, List
import ipywidgets as widgets
import numpy as np
import math

MOVE_ACTION = 'move'
ZOOM_ACTION = 'zoom'
ROTATE_ACTION = 'rotate'
SELECT_ACTION = 'select'

output = Output()

def defaultColorMap(cells, variables):
    return [
        ('#004400', 'green', cells[variables['cycle_model']] == 5),
        ('brown', 'black', cells[variables['cycle_model']] == 100),
        ('#440000', 'red', cells[variables['cycle_model']] == 101)
    ]

def count(array):
    dic = {}
    for num in array:
        if not num in dic:
            dic[num] = 0
        dic[num] += 1
    return dic
    

class Interactor3D:
    def __init__(self, parser, width: int = 500, height: int = 400, colorMap = defaultColorMap):
        self._currentFrame = parser.getFrameRange()[0]
        self._canvas = Canvas(width=width, height=height)
        self._parser = parser
        self._colorMap = colorMap
        self._height = height
        self._width = width
        
        frame = parser.getFrame(self._currentFrame)
        mesh = frame.environment.mesh
        self._zoom = max((mesh.boundsX[1] - mesh.boundsX[0]) / width, (mesh.boundsY[1] - mesh.boundsY[0]) / height)
        
        self._xOffset = mesh.boundsX[0]
        self._yOffset = mesh.boundsY[0]
        self._selectedCell = None
        
        self._clicking = False
        self._dragStartX = 0
        self._dragStartY = 0
        self._actionOriginX = 0
        self._actionOriginY = 0
        
        self._canvas.on_mouse_down(self.onMouseDown)
        self._canvas.on_mouse_move(self.onMouseMove)
        self._canvas.on_mouse_up(self.onMouseUp)
        self._canvas.on_mouse_out(self.onMouseOut)
        
        self.action = MOVE_ACTION
        
        self._buttons = widgets.RadioButtons(
            options=[MOVE_ACTION, ROTATE_ACTION, ZOOM_ACTION, SELECT_ACTION],
            value=self.action,
            description='Mouse Action:',
            disabled=False,
        )
        self._buttons.observe(self.onToolChange, names='value')
        
        self._frameSelector = widgets.IntSlider(
            value=self._currentFrame,
            min=parser.getFrameRange()[0],
            max=parser.getFrameRange()[1] - 1,
            step=1,
            description='Frame:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self._frameSelector.observe(self.onFrameChange, names='value')
        
        self.previousFrameNumber = -1
        self.LastPickedActor = None
        self.LastPickedProperty = vtk.vtkProperty()
        
        self.renderer = vtk.vtkRenderer()
        
        self.create(frame)
        
        renderWindow = vtk.vtkRenderWindow()
        renderWindow.SetOffScreenRendering(1)
        renderWindow.AddRenderer(self.renderer)
        renderWindow.SetSize(width, height)
        renderWindow.Render()
        
        self.renderWindow = renderWindow
        
        self.selectCell(width/4,height/2)
        
        self.update()
    
    def create(self, frameNumber):
        if frameNumber == self.previousFrameNumber:
            return
        self.previousFrameNumber = frameNumber
        frame = self._parser.getFrame(self._currentFrame)
        
        self.renderer.RemoveAllViewProps()
        cells = frame.cells.data
        variables = frame.cells.variables
        x = variables["position.x"]
        y = variables["position.y"]
        z = variables["position.z"]
        r = self.radiusOfCells(cells, variables)
        for cell in range(cells.shape[1]):
            source = vtk.vtkSphereSource()

            radius = r[cell]

            source.SetRadius(radius)
            source.SetCenter(cells[x,cell], cells[y,cell], cells[z, cell])
            source.SetPhiResolution(11)
            source.SetThetaResolution(21)

            mapper = vtk.vtkPolyDataMapper()
            mapper.SetInputConnection(source.GetOutputPort())
            actor = vtk.vtkActor()
            actor.SetMapper(mapper)

            red = vtk.vtkMath.Random(.4, 1.0)
            g = vtk.vtkMath.Random(.4, 1.0)
            b = vtk.vtkMath.Random(.4, 1.0)
            actor.GetProperty().SetDiffuseColor(red, g, b)
            actor.GetProperty().SetDiffuse(.8)
            actor.GetProperty().SetSpecular(.5)
            actor.GetProperty().SetSpecularColor(1.0, 1.0, 1.0)
            actor.GetProperty().SetSpecularPower(30.0)

            self.renderer.AddActor(actor)
        
    
    def update(self):
        canvas = self._canvas
        canvas.clear()
        
        self.create(self._currentFrame)
        
        self.renderWindow.Render()
        
        windowToImageFilter = vtk.vtkWindowToImageFilter()
        windowToImageFilter.SetInput(self.renderWindow)
        windowToImageFilter.Update()

        writer = vtk.vtkPNGWriter()
        writer.SetWriteToMemory(1)
        writer.SetInputConnection(windowToImageFilter.GetOutputPort())
        writer.Write()
        
        data = memoryview(writer.GetResult()).tobytes()
        
        image = Image(value=data)
        
        canvas.draw_image(image)
            
        
        canvas.fill_style = '#A0A0A0'
        canvas.font = '10px serif'
        canvas.fill_text(f"(x: {self._xOffset}, y: {self._yOffset}, s: {self._zoom}, c: {self._selectedCell})", 10, self._height - 10)
        
    
    @output.capture()
    def onMouseDown(self, x: int, y: int):
        self._dragStartX = x
        self._dragStartY = y
        self._envActionOriginX = x * self._zoom + self._xOffset
        self._envActionOriginY = y * self._zoom + self._yOffset
        self._actionOriginX = x
        self._actionOriginY = y
        self._actionOriginZoom = self._zoom
        
        if self.action == SELECT_ACTION:
            self.selectCell(x, self._height - y)
        else:
            self._clicking = True
    
    def onMouseUp(self, x: int, y: int):
        self._clicking = False
    
    def onMouseOut(self, x: int, y: int):
        self._clicking = False
    
    @output.capture()
    def onMouseMove(self, x: int, y: int):
        if self._clicking:
            if self.action == MOVE_ACTION:
                self.pan(x, y)
            elif self.action == ZOOM_ACTION:
                self.renderer.GetActiveCamera().Zoom(1 + (self._actionOriginY - y) / 100)
            elif self.action == ROTATE_ACTION:
                self.rotate(x, y)
                
            self._dragStartX = x
            self._dragStartY = y
            self.update()
    
    def pan(self, x: float, y: float):
        # from https://compucell3d.org/BinDoc/cc3d_binaries/dependencies/windows/MinGW/dependencies_qt_4.8.4_pyqt_4.9.6_vtk_5.10.1_python27/Player/vtk/wx/wxVTKRenderWindow.py
        renderer = self.renderer
        camera = renderer.GetActiveCamera()
        (pPoint0,pPoint1,pPoint2) = camera.GetPosition()
        (fPoint0,fPoint1,fPoint2) = camera.GetFocalPoint()

        renderer.SetWorldPoint(fPoint0,fPoint1,fPoint2,1.0)
        renderer.WorldToDisplay()
        # Convert world point coordinates to display coordinates
        dPoint = renderer.GetDisplayPoint()
        focalDepth = dPoint[2]

        aPoint0 = self._width / 2 + (x - self._dragStartX)
        aPoint1 = self._height / 2 - (y - self._dragStartY)

        renderer.SetDisplayPoint(aPoint0,aPoint1,focalDepth)
        renderer.DisplayToWorld()

        (rPoint0,rPoint1,rPoint2,rPoint3) = renderer.GetWorldPoint()
        if (rPoint3 != 0.0):
            rPoint0 = rPoint0/rPoint3
            rPoint1 = rPoint1/rPoint3
            rPoint2 = rPoint2/rPoint3

        camera.SetFocalPoint((fPoint0 - rPoint0) + fPoint0,
                             (fPoint1 - rPoint1) + fPoint1,
                             (fPoint2 - rPoint2) + fPoint2)

        camera.SetPosition((fPoint0 - rPoint0) + pPoint0,
                           (fPoint1 - rPoint1) + pPoint1,
                           (fPoint2 - rPoint2) + pPoint2)
        
    def rotate(self, x: float, y: float):
        # from https://compucell3d.org/BinDoc/cc3d_binaries/dependencies/windows/MinGW/dependencies_qt_4.8.4_pyqt_4.9.6_vtk_5.10.1_python27/Player/vtk/wx/wxVTKRenderWindow.py
        renderer = self.renderer
        camera = renderer.GetActiveCamera()
        camera.Azimuth(self._dragStartX - x)
        camera.Elevation(y - self._dragStartY)
        camera.OrthogonalizeViewUp()

        renderer.ResetCameraClippingRange()
    
    def zoom(self, x: float, y: float):
        # from https://compucell3d.org/BinDoc/cc3d_binaries/dependencies/windows/MinGW/dependencies_qt_4.8.4_pyqt_4.9.6_vtk_5.10.1_python27/Player/vtk/wx/wxVTKRenderWindow.py
        
        renderer = self.renderer
        camera = renderer.GetActiveCamera()

        zoomFactor = math.pow(1.02,(0.5*(self._dragStartY - y)))
        self._CurrentZoom = self._CurrentZoom * zoomFactor

        if camera.GetParallelProjection():
            parallelScale = camera.GetParallelScale()/zoomFactor
            camera.SetParallelScale(parallelScale)
        else:
            camera.Dolly(zoomFactor)
            renderer.ResetCameraClippingRange()

        self._dragStartX = x
        self._dragStartY = y

        self.Render()
    
    
        
    
    def onToolChange(self, action):
        self.action = action.new
    
    def onFrameChange(self, action):
        self._currentFrame = action.new
        self.update()
    
    @output.capture()
    def selectCell(self, x: float, y: float):
        picker = vtk.vtkPropPicker()
        picker.Pick(x, y, 0, self.renderer)

        # get the new
        self.NewPickedActor = picker.GetActor()
        
        position = picker.GetPickPosition()
        
        if not picker.GetActor():
            self._selectedCell = None
            self.update()
            return
        
        x, z, y = picker.GetPickPosition()

        # If something was selected
        if self.NewPickedActor:
            # If we picked something before, reset its property
            if self.LastPickedActor:
                self.LastPickedActor.GetProperty().DeepCopy(self.LastPickedProperty)

            # Save the property of the picked actor so that we can
            self.LastPickedProperty.DeepCopy(self.NewPickedActor.GetProperty())
            # Highlight the picked actor by changing its properties
            self.NewPickedActor.GetProperty().SetColor(0.0, 0.0, 0.0)
            self.NewPickedActor.GetProperty().SetDiffuse(0.0)
            self.NewPickedActor.GetProperty().SetSpecular(0.0)
            
            self.LastPickedActor = self.NewPickedActor
            
        frame = self._parser.getFrame(self._currentFrame)
    
        cells = frame.cells.data
        cellVariables = frame.cells.variables
        
        distances = np.sqrt(np.square(x - cells[cellVariables['position.x']]) + np.square(y - cells[cellVariables['position.y']]) + np.square(z - cells[cellVariables['position.z']])) - self.radiusOfCells(cells, cellVariables)
        minIndex = np.argmin(distances)
        print(minIndex)
        
        if distances[minIndex] <= 0:
            self._selectedCell = cells[cellVariables['ID'], minIndex]
            
        self.update()
        
        
 
    def radiusOfCells(self, cells, variables):
        return (cells[variables['total_volume']] * (3 / ( 4 * math.pi))) ** (1 / 3)

    def show(self):
        display(self._canvas, self._buttons, self._frameSelector, output)
            



def InteractiveEnvironment(outputPath: str, width: int = 500, height: int = 400):
    parser = Parser(outputPath)
    if parser.getFrame(parser.getFrameRange()[0]).environment.is2D:
        return Interactor2D(parser, width, height)

In [81]:
Interactor3D(Parser('./sample-output')).show()

Canvas(height=400, width=500)

RadioButtons(description='Mouse Action:', options=('move', 'rotate', 'zoom', 'select'), value='move')

IntSlider(value=0, continuous_update=False, description='Frame:', max=9)

Output()